In [ ]:
from scipy.io.wavfile import write
import numpy as np, os

os.makedirs("tones", exist_ok=True)
def gen(freq, freq_label, vol):
    t = np.linspace(0, 2, 44100*2, False)
    tone = vol * np.sin(2*np.pi*freq*t)
    arr = np.int16(tone * 32767)
    write(f"tones/{freq}Hz_{int(vol*100)}dB.wav", 44100, arr)

for f in [250,500,1000,2000,4000,8000]:
    for v in [0.2,0.4]:
        gen(f, f, v)